In [6]:
import pandas as pd
import os

# Définir l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

In [7]:
os.getcwd()

'/workspaces/projet-immo/notebooks'

In [8]:
fonciere_data = pd.read_csv("/workspaces/projet-immo/data/fonciere.csv",sep=";")

Liste des acronymes  

**CA** : Communauté d’agglomération  
**CC** : Communauté de communes  
**CU** : Communauté urbaine  
**MET** : Métropole  
**TEOM** : Taxe d’enlèvement des ordures ménagères  
**TFB** : Taxe foncière sur les propriétés bâties  
**TFNB** : Taxe foncière sur les propriétés non bâties  
**TH** : Taxe d’habitation  
**THRS** : Taxe d’habitation sur les résidences secondaires  
**THLV** : Taxe d’habitation sur les logements vacants  

In [12]:
colonnes_etude = ["EXERCICE","REG","LIBREG","DEP","LIBDEP","COM","LIBCOM","Taux_Global_TFNB","Taux_Global_TFB"]

In [19]:
fonciere_data.loc[(fonciere_data["EXERCICE"]==2023),colonnes_etude].head()

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,Taux_Global_TFNB,Taux_Global_TFB
1131,2023,44,GRAND-EST,55,MEUSE,113,CHEPPY,71.06,43.20
1132,2023,44,GRAND-EST,55,MEUSE,133,COUVERTPUIS,86.67,43.78
1133,2023,44,GRAND-EST,55,MEUSE,143,DAMLOUP,89.61,50.34
1134,2023,44,GRAND-EST,55,MEUSE,163,DONCOURT AUX TEMPLIERS,83.07,45.23
1135,2023,44,GRAND-EST,55,MEUSE,168,DUZEY,75.37,47.48


In [18]:
fonciere_data.loc[(fonciere_data["DEP"]=="75")&(fonciere_data["EXERCICE"]==2023),colonnes_etude]

,EXERCICE,REG,LIBREG,DEP,LIBDEP,COM,LIBCOM,Taux_Global_TFNB,Taux_Global_TFB
21143,2023,11,ILE-DE-FRANCE,75,PARIS,56,VILLE DE PARIS,28.86,21.15


Formule de calcul de la taxe fonciere

Taxe foncière = (Valeur locative cadastrale×50%) × Taux d’imposition

ValeurLocativeCadastrale = Surface pondérée x Tarif au m² x Coefficient de localisation x Coefficient d'entretien

In [23]:
1100*12*0.5*0.2115

1395.8999999999999